In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import torch
from torch import nn
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer
from transformers import BertModel
from datasets import Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import f1_score
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
from sklearn import metrics
import torch.nn.functional as F
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
from util import *
import torch.nn.functional as F
from datetime import datetime
import pprint
import os
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
import util
#from catalyst.metrics.functional import process_multilabel_components
#from catalyst.metrics import multi_label_accuracy
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.core.decorators import auto_move_data
from sklearn.metrics import label_ranking_average_precision_score, accuracy_score, f1_score
#from ignite.utils import convert_tensor
#from pytorch_metric_learning.miners import MultiSimilarityMiner
#from pytorch_metric_learning.losses import TripletMarginLoss
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    )

In [3]:
def tokenize(tokenizer, text, label, max_len):
        text = str(text)
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            label,
            truncation=True,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            f"input_ids": torch.tensor(ids, dtype=torch.long),
            f"attention_mask": torch.tensor(mask, dtype=torch.long),
            f"token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
        }
    

class PatentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, abstract_max_len=160, claims_max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.claims_max_len = claims_max_len
        self.abstract_max_len = abstract_max_len

        self.claims = dataframe.claims
        self.abstracts = dataframe.abstract        
        self.labels = dataframe.labels


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        
        abstract = self.abstracts[index] #self.tokenize(self.abstracts[index], max_len=self.abstract_max_len)
        claims = self.claims[index] #self.tokenize(self.claims[index], max_len=self.claims_max_len)
        labels = self.labels[index]
        #embedded_cpc = torch.tensor(np.array(self.data.embedded_cpc[index]), dtype=torch.float)
        
        #labels = torch.tensor(self.labels[index])
        return {"abstract": abstract, 
                "claims": claims,
                #'embedded_cpc': embedded_cpc,
                'labels': labels}
    


In [4]:
from sklearn.metrics import classification_report, f1_score

In [5]:
# import torch.nn.functional as F
# def linear_combination(x, y, epsilon): 
#     return epsilon*x + (1-epsilon)*y

# def reduce_loss(loss, reduction='mean'):
#     return loss.mean() if reduction=='mean' else loss.sum() if reduction=='sum' else loss

# class LabelSmoothingCrossEntropy(nn.Module):
#     def __init__(self, pos_weights, epsilon:float=0.1, reduction='mean'):
#         super().__init__()
#         self.epsilon = epsilon
#         self.reduction = reduction
#         self.pos_weights = pos_weights
    
#     def forward(self, preds, target):
#         n = preds.size()[-1]
#         #log_preds = F.log_softmax(preds, dim=-1)
#         loss = reduce_loss(-preds.sum(dim=-1), self.reduction)
#         nll = F.binary_cross_entropy_with_logits(preds, target, pos_weight=self.pos_weights.to("cuda:0"), reduction=self.reduction)
#         return linear_combination(loss/n, nll, self.epsilon)

In [6]:
from pytorch_lightning.metrics import functional as FM


class BasicSystem(pl.LightningModule):
    def __init__(self, 
                     batch_size=16, 
                     model_name="bertForPatents/", 
                     gradient_checkpointing=True, 
                     learning_rate = 3e-5, 
                     subset=list(sorted(set(all_tiers_100)-set(["PersonalizedProduct"])))):
        super().__init__()
        self.model_name = model_name
        self.subset = subset
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.gradient_checkpointing = gradient_checkpointing
        
    def prepare_data(self):
        self.training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
        self.testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        
#         self.cpc_embeddings = np.fromfile("/home/martin/patentmark/cpc.node2vec.emb.32d.bin", dtype=np.float32).reshape((-1,32))
#         self.cpc_labelizer = joblib.load('./node2id.joblib')
#         self.cpc_lookup = {c: n for n, c in enumerate(self.cpc_labelizer.classes_)}

#     @f.collecting
#     def convert_cpc_codes(self, codes):
#         for code in codes:
#             if code in self.cpc_lookup:
#                 yield self.cpc_lookup[code]
    
#     def embed_cpc_codes(self,codes):
#         embedding = np.zeros(32)
#         converted = self.convert_cpc_codes(codes)

#         if not converted:
#             return embedding

#         for code_id in converted:
#             embedding = embedding + self.cpc_embeddings[code_id]

#         return embedding / len(converted)
    
        
    def setup(self,stage):

        training_labels = self.training_set[self.subset].apply(util.array_labels, axis=1)
        testing_labels = self.testing_set[self.subset].apply(util.array_labels, axis=1)
        
        self.training_set['labels'] = training_labels
        self.testing_set['labels'] = testing_labels
        
        #self.training_set['embedded_cpc'] = self.training_set.cpc_codes.apply(self.embed_cpc_codes)
        #self.testing_set['embedded_cpc'] = self.testing_set.cpc_codes.apply(self.embed_cpc_codes)
        
        self.training_dataset = PatentDataset(self.training_set, self.tokenizer)
        self.testing_dataset = PatentDataset(self.testing_set, self.tokenizer)
        
        self.setup_classifier()
        

    def test_label(self, text, label):
        nice_label = tier_translations[label]
        inputs = self.tokenizer.encode_plus(
            text,
            nice_label,
            truncation=True,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        cont_reps, pooler_output = self.text_embedder(inputs['input_ids'], inputs['attn_masks'], inputs['token_type_ids'])
        logits = self.cls_layer(self.dropout(pooler_output))

    def setup_classifier(self):
        self.text_embedder = AutoModel.from_pretrained(self.model_name, gradient_checkpointing=self.gradient_checkpointing)
        self.cls_layer = nn.Linear(1024, 1)
        self.dropout = nn.Dropout(0.1)

    
    @auto_move_data
    def forward(self, abstract, claims):
        
        
        for label in subset:
            predictions
            
        x = self.embed_patent(abstract, claims, embedded_cpc)
        x = self.classify_patent(x)
        return x
        
    def train_dataloader(self):
        return DataLoader(self.training_dataset, batch_size=self.batch_size, shuffle=True, num_workers=32)

    def val_dataloader(self):
        return DataLoader(self.testing_dataset, batch_size=self.batch_size, shuffle=False, num_workers=32)

    def test_dataloader(self):
        return self.val_dataloader()
    
    def training_step(self, batch, batch_idx):
        abstract = batch['abstract']
        claims = batch['claims']
        embedded_cpc = batch['embedded_cpc']
        labels = batch['labels']
        
        embedding = self.embed_patent(abstract, claims, embedded_cpc)
        logits = self.classify_patent(embedding)
        loss = self.loss_function(logits, labels)
        self.log("traing_loss", loss, prog_bar=True)
 
        return loss
    
    
    def validation_step(self, batch, batch_idx):
        abstract = batch['abstract']
        claims = batch['claims']
        embedded_cpc = batch['embedded_cpc']
        labels = batch['labels']
        
    
        embedding = self.embed_patent(abstract, claims, embedded_cpc)
        logits = self.classify_patent(embedding)
        loss = self.loss_function(logits, labels)
        self.log("val_loss", loss, prog_bar=True)
        
        predictions = (F.sigmoid(logits).cpu().detach().numpy() >= 0.5).tolist()
        labels = labels.cpu().detach().numpy().astype(np.bool).tolist()
        
        return predictions, labels
    
    def validation_epoch_end(self, outputs):
        predictions = list(f.cat(x[0] for x in outputs))
        labels = list(f.cat(x[1] for x in outputs))
        self.log("val_f1", f1_score(labels, predictions, average="samples"), prog_bar=True)
        print(classification_report(labels, predictions, target_names=self.label_encoder.classes_))
        
    

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)

        

In [7]:
model = BasicSystem()
early_stopping = EarlyStopping('val_f1', mode="max", verbose=True, patience=3)
#early_stopping = EarlyStopping('loss', mode="min", verbose=True, patience=3)
trainer = pl.Trainer(gpus=1,
                     #overfit_batches=5,
                     callbacks=[early_stopping],
                     #precision=16,
                     #auto_scale_batch_size=True,
                     #auto_lr_find=True,
                     log_every_n_steps=5,
                     #val_check_interval=100000,
                     #limit_val_batches=0.0,
                     flush_logs_every_n_steps=5)


GPU available: True, used: True
2020-12-07 07:01:20 - GPU available: True, used: True
TPU available: False, using: 0 TPU cores
2020-12-07 07:01:20 - TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2020-12-07 07:01:20 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [8]:
#trainer.tune(model)

In [9]:
trainer.fit(model)


  | Name                        | Type              | Params
------------------------------------------------------------------
0 | text_embedder               | BertModel         | 344 M 
1 | embedding_dropout_layer     | Dropout           | 0     
2 | embedding_layer             | Linear            | 1.1 M 
3 | preclassifier_dropout_layer | Dropout           | 0     
4 | preclassifier_layer         | Linear            | 131 K 
5 | classifier_layer            | Linear            | 5.7 K 
6 | loss_function               | BCEWithLogitsLoss | 0     
2020-12-07 07:01:30 - 
  | Name                        | Type              | Params
------------------------------------------------------------------
0 | text_embedder               | BertModel         | 344 M 
1 | embedding_dropout_layer     | Dropout           | 0     
2 | embedding_layer             | Linear            | 1.1 M 
3 | preclassifier_dropout_layer | Dropout           | 0     
4 | preclassifier_layer         | Linear         

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.00      0.00      0.00        14
           Analysis and Modeling: 3D Modeling       0.27      0.36      0.31        11
                            Anatomical Target       0.00      0.00      0.00        22
           Anatomical Target: Lower Extremity       0.48      1.00      0.65        15
     Anatomical Target: Lower Extremity - Hip       0.16      1.00      0.27         5
    Anatomical Target: Lower Extremity - Knee       0.00      0.00      0.00        10
                     Anatomical Target: Torso       0.00      0.00      0.00         2
             Anatomical Target: Torso - Spine       0.00      0.00      0.00         1
           Anatomical Target: Upper Extremity       0.16      1.00      0.27         5
Anatomical Target: Upper Extremity - Shoulder       0.16      1.00      0.28         5
                                      Imag

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.51        84
           Analysis and Modeling: 3D Modeling       0.29      1.00      0.45        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.16      1.00      0.28        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.14      0.97      0.25        35
             Anatomical Target: Torso - Spine       0.08      0.86      0.15        21
           Anatomical Target: Upper Extremity       0.13      0.90      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.00      0.00      0.00        23
                                      Imag

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.51        84
           Analysis and Modeling: 3D Modeling       0.29      1.00      0.45        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.17      0.75      0.28        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.14      0.97      0.25        35
             Anatomical Target: Torso - Spine       0.08      0.81      0.15        21
           Anatomical Target: Upper Extremity       0.13      1.00      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.09      1.00      0.17        23
                                      Imag

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.51        84
           Analysis and Modeling: 3D Modeling       0.29      0.97      0.45        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.16      1.00      0.28        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.00      0.00      0.00        35
             Anatomical Target: Torso - Spine       0.00      0.00      0.00        21
           Anatomical Target: Upper Extremity       0.13      0.84      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.10      1.00      0.17        23
                                      Imag

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.35      1.00      0.51        84
           Analysis and Modeling: 3D Modeling       0.29      1.00      0.45        71
                            Anatomical Target       0.67      1.00      0.81       164
           Anatomical Target: Lower Extremity       0.47      1.00      0.63       113
     Anatomical Target: Lower Extremity - Hip       0.16      1.00      0.28        40
    Anatomical Target: Lower Extremity - Knee       0.34      1.00      0.50        82
                     Anatomical Target: Torso       0.14      0.97      0.25        35
             Anatomical Target: Torso - Spine       0.08      0.90      0.15        21
           Anatomical Target: Upper Extremity       0.13      1.00      0.23        31
Anatomical Target: Upper Extremity - Shoulder       0.10      1.00      0.18        23
                                      Imag

1

In [10]:
trainer.test()

1

In [11]:
# def stack(listOfDicts):
#     initDict = listOfDicts[0]
#     finalDict = {}
#     for key in initDict.keys():
#         tensors = tuple(d[key] for d in listOfDicts)
#         finalDict[key] = torch.stack(tensors)
#     return finalDict

In [12]:
# abstracts = stack(list(tokenize(tokenizer, x, max_len=MAX_LEN_ABSTRACT) for x in testing_set.abstract))

# claims = stack(list(tokenize(tokenizer, x, max_len=MAX_LEN_CLAIMS) for x in testing_set.claims))

# predictions = model.forward(abstract=abstracts, claims=claims)

# binarized = predictions.detach().numpy() > 0.5

# from sklearn.metrics import classification_report
# testing_labels = testing_set[subset]
# print(classification_report(testing_labels, binarized))